**Nombre:** Laura Melissa Barrera Pinto

**Código:** 202422789

## Instalación de FastAPI

In [1]:
pip install --user --upgrade --force-reinstall fastapi

  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached starlette-0.46.1-py3-none-any.whl.metadata (6.2 kB)
  Using cached pydantic-2.11.0-py3-none-any.whl.metadata (63 kB)
  Using cached typing_extensions-4.13.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.0-cp312-cp312-win_amd64.whl.metadata (6.9 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached fastapi-0.115.12-py3-none-any.whl (95 kB)
Using cached pydantic-2.11.0-py3-none-any.whl (442 kB)
Using cached pydantic_core-2.33.0-cp312-cp312-win_amd64.whl (2.0 MB)
Using cached starlette-0.46.1-py3-none-any.whl (71 kB)
Using cached typing_extensions-4.13.0-py3-none-any.whl (45 kB)
Using cached annotat


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Selección de modelos para el despliegue

Se realizará una API para la clasificación de Churn de los clientes de una compañía de telecomunicaciones, teniendo en cuenta los siguientes datos clave:
- Call Failure (Numero fallos de llamada)
- Complaints (0: Sin queja, 1 Queja)
- Subscription Length (Meses de suscripción)
- Charge Amount (0: Mas bajo, 9: Más alto)
- Seconds of use (Total de segundos)
- Frecuency of use (Numero total llamadas)
- Frecuency of SMS (Numero total de mensajes de texto)
- Distinct Called Numbers (Numero total llamadas distintas)
- Age group (1: Más joven, 5: Más viejo)
- Status (1: Activo, 2: No activo)
- Age (Edad del cliente)
- Customer Value (Valor del plan)
- Churn (1: abandono, 0: No abandono)

Se aplicara el modelo de RandomForest y Regresión Logistica

In [2]:
# Librerias para preprocesamiento y entrenamiento del modelo
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Librerias para API REST
import joblib
import logging
import uvicorn
import threading
import requests
from fastapi import FastAPI
from pydantic import BaseModel

In [3]:
# Cargue del dataset y seleccion de variables
data = pd.read_csv(r'C:\Users\User\Downloads\Analitica para la toma de decisiones\Actividades\Unidad 3\Customer Churn.csv')
X = data.drop(columns=["Churn"])
y = data["Churn"]

## Registro de eventos (loggin)

Se realizara el registro de eventos desde el entrenamiento del modelo

In [4]:
# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

## Entrenamiento del modelo

El entrenamiento del modelo se realiza de manera normal y se almacenan los modelos entrenados para su posterior despliegue en la API

In [5]:
# División en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Estandarización de variables numéricas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
# Guardar el scaler que se uso para los datos en el modelo
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

### Random Forest

In [7]:
# Definir el espacio de hiperparámetros a explorar
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

In [8]:
# Entrenamiento del modelo
rf_model = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring="f1", n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Guardar los mejores parámetros
logger.info(f"Mejor configuración de hiperparámetros Random Forest: {grid_search.best_params_}")

# Entrenar el mejor modelo encontrado
best_rf_model = grid_search.best_estimator_

2025-03-28 15:50:19,280 - INFO - Mejor configuración de hiperparámetros Random Forest: {'max_depth': 30, 'min_samples_split': 5, 'n_estimators': 100}


In [9]:
# Predicción en conjunto de prueba
y_pred = best_rf_model.predict(X_test_scaled)
logger.info(f"Precisión del modelo Random Forest entrenado: {accuracy_score(y_test, y_pred):.2f}")

# Guardar el modelo
try:
    joblib.dump(best_rf_model, "modelo_churn_rf.pkl")
    logger.info("Modelo de Churn Random Forest guardado exitosamente.")
except Exception as e:
    logger.error(f"Error al guardar el modelo Random Forest: {e}")

2025-03-28 15:50:19,355 - INFO - Precisión del modelo Random Forest entrenado: 0.93
2025-03-28 15:50:19,467 - INFO - Modelo de Churn Random Forest guardado exitosamente.


### Regresión Logistica

In [10]:
# Entrenar el modelo de Regresión Logística
log_model = LogisticRegression(max_iter=1000, random_state=42)
log_model.fit(X_train_scaled, y_train)

# Predicciones
y_pred_log = log_model.predict(X_test_scaled)
logger.info(f"Precisión del modelo Regresión Logistica entrenado: {accuracy_score(y_test, y_pred_log):.2f}")

# Guardar el modelo
try:
    joblib.dump(log_model, "modelo_churn_log.pkl")
    logger.info("Modelo de Churn Regresión Logistica guardado exitosamente.")
except Exception as e:
    logger.error(f"Error al guardar el modelo Regresión Logistica: {e}")

2025-03-28 15:50:19,574 - INFO - Precisión del modelo Regresión Logistica entrenado: 0.87
2025-03-28 15:50:19,578 - INFO - Modelo de Churn Regresión Logistica guardado exitosamente.


## Despliegue del modelo

El modelo se despliega a través de una API de manera local

In [11]:
# Inicializar FastAPI
app = FastAPI(title="API de Clasificación de Churn")

# Definir el modelo de entrada
class ChurnInput(BaseModel):
    Call_Failure: int
    Complaints: int
    Subscription_Length: int
    Charge_Amount: int
    Seconds_of_use: int
    Frecuency_of_use: int
    Frecuency_of_SMS: int
    Distinct_Called_Numbers: int
    Age_group: int
    Tariff_Plan: int
    Status: int
    Age: int
    Customer_Value: float

In [12]:
# Cargar modelos
scaler = joblib.load("scaler.pkl")
try:
    model_rf = joblib.load("modelo_churn_rf.pkl")
    model_log = joblib.load("modelo_churn_log.pkl")
    logger.info("Modelo cargado exitosamente en la API.")
except Exception as e:
    logger.error(f"Error al cargar el modelo en la API: {e}")
    model_rf = None
    model_log = None

2025-03-28 15:50:19,722 - INFO - Modelo cargado exitosamente en la API.


### Definir acceso a los modelos

In [13]:
# Definir URL para acceso

@app.post("/predict_rf")
async def predict_model_rf(data: ChurnInput):
    """Recibe datos del cliente y devuelve la clasificación de churn."""
    datos = np.array([data.Call_Failure, data.Complaints, data.Subscription_Length, data.Charge_Amount, data.Seconds_of_use, data.Frecuency_of_use, data.Frecuency_of_SMS, data.Distinct_Called_Numbers, data.Age_group, data.Tariff_Plan, data.Status, data.Age, data.Customer_Value]).reshape(1, -1)
    
    try:
        data_df = pd.DataFrame(datos, columns=X.columns)
        input_scaled = scaler.transform(data_df)  
        prediction = model_rf.predict(input_scaled)[0]
        probability = model_rf.predict_proba(input_scaled)[0][1]
        logger.info(f"Predicción de Churn Random Forest: {int(prediction)}, Probabilidad de Churn Random Forest: {round(probability,2)}")
        return {"Predicción de Churn Random Forest": int(prediction), "Probabilidad de Churn Random Forest": round(probability,2)}
    except Exception as e:
        logger.error(f"Error en la predicción Random Forest: {e}")
        return {"error": str(e)}
    

@app.post("/predict_log")
async def predict_model_log(data: ChurnInput):
    """Recibe datos del cliente y devuelve la clasificación de churn."""
    datos = np.array([data.Call_Failure, data.Complaints, data.Subscription_Length, data.Charge_Amount, data.Seconds_of_use, data.Frecuency_of_use, data.Frecuency_of_SMS, data.Distinct_Called_Numbers, data.Age_group, data.Tariff_Plan, data.Status, data.Age, data.Customer_Value]).reshape(1, -1)
    
    try:
        data_df = pd.DataFrame(datos, columns=X.columns)
        input_scaled = scaler.transform(data_df)  
        prediction = model_log.predict(input_scaled)[0]
        probability = model_log.predict_proba(input_scaled)[0][1]
        logger.info(f"Predicción de Churn Regresión Logistica: {int(prediction)}, Probabilidad de Churn Regresión Logistica: {round(probability,2)}")
        return {"Predicción de Churn Regresión Logistica": int(prediction), "Probabilidad de Churn Regresión Logistica": round(probability,2)}
    except Exception as e:
        logger.error(f"Error en la predicción Regresión Logistica: {e}")
        return {"error": str(e)}

## Ejecutar API en segundo plano

In [ ]:
# Ejecutar API en segundo plano
def run_api():
  logger.info("Iniciando servidor FastAPI...")
  uvicorn.run(app, host="0.0.0.0", port=8000)

threading.Thread(target=run_api, daemon=True).start()

2025-03-28 15:51:36,107 - INFO - Iniciando servidor FastAPI...


INFO:     Started server process [8364]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
2025-03-28 15:52:05,964 - INFO - Predicción de Churn Random Forest: 1, Probabilidad de Churn Random Forest: 0.9


INFO:     127.0.0.1:42087 - "POST /predict_rf HTTP/1.1" 200 OK


2025-03-28 15:52:11,644 - INFO - Predicción de Churn Random Forest: 1, Probabilidad de Churn Random Forest: 0.82


INFO:     127.0.0.1:42089 - "POST /predict_rf HTTP/1.1" 200 OK


...

INFO:     127.0.0.1:42236 - "POST /predict_log HTTP/1.1" 200 OK


2025-03-28 16:03:08,401 - INFO - Predicción de Churn Regresión Logistica: 0, Probabilidad de Churn Regresión Logistica: 0.26


INFO:     127.0.0.1:42237 - "POST /predict_log HTTP/1.1" 200 OK


2025-03-28 16:03:08,412 - INFO - Predicción de Churn Regresión Logistica: 0, Probabilidad de Churn Regresión Logistica: 0.05


INFO:     127.0.0.1:42238 - "POST /predict_log HTTP/1.1" 200 OK


## Casos de prueba del modelo sin Postman

Este codigo solo se debe ejecutar cuando no se tenga acceso a Postman para probar la URL.

In [15]:
# Pruebas de la API
def test_api_rf():
    sample_no_churn = {
        "Call_Failure": 0,
        "Complaints": 0,
        "Subscription_Length": 24,
        "Charge_Amount": 50,
        "Seconds_of_use": 300,
        "Frecuency_of_use": 10,
        "Frecuency_of_SMS": 5,
        "Distinct_Called_Numbers": 3,
        "Age_group": 2,
        "Tariff_Plan": 1,
        "Status": 1,
        "Age": 35,
        "Customer_Value": 200.0
    }
    
    sample_churn = {
        "Call_Failure": 2,
        "Complaints": 3,
        "Subscription_Length": 6,
        "Charge_Amount": 20,
        "Seconds_of_use": 100,
        "Frecuency_of_use": 3,
        "Frecuency_of_SMS": 1,
        "Distinct_Called_Numbers": 1,
        "Age_group": 1,
        "Tariff_Plan": 2,
        "Status": 0,
        "Age": 22,
        "Customer_Value": 50.0
    }

    sample_Data_churn = {
        "Call_Failure": 2,
        "Complaints": 0,
        "Subscription_Length": 41,
        "Charge_Amount": 0,
        "Seconds_of_use": 628,
        "Frecuency_of_use": 13,
        "Frecuency_of_SMS": 17,
        "Distinct_Called_Numbers": 5,
        "Age_group": 4,
        "Tariff_Plan": 1,
        "Status": 2,
        "Age": 45,
        "Customer_Value": 58.525
    }

    sample_Data_no_churn = {
        "Call_Failure": 8,
        "Complaints": 0,
        "Subscription_Length": 38,
        "Charge_Amount": 0,
        "Seconds_of_use": 4370,
        "Frecuency_of_use": 71,
        "Frecuency_of_SMS": 5,
        "Distinct_Called_Numbers": 17,
        "Age_group": 3,
        "Tariff_Plan": 1,
        "Status": 1,
        "Age": 30,
        "Customer_Value": 197.64
    }
    
    url = "http://127.0.0.1:8000/predict_rf"
    
    response_no_churn = requests.post(url, json=sample_no_churn)
    logger.info(f"Respuesta para cliente sin churn: {response_no_churn.json()}")
    
    response_churn = requests.post(url, json=sample_churn)
    logger.info(f"Respuesta para cliente con churn: {response_churn.json()}")

    response_churn_org = requests.post(url, json=sample_Data_churn)
    logger.info(f"Respuesta orginal cliente con churn: {response_churn_org.json()}")

    response_no_churn_org = requests.post(url, json=sample_Data_no_churn)
    logger.info(f"Respuesta original cliente sin churn: {response_no_churn_org.json()}")

test_api_rf()

def test_api_log():
    sample_no_churn = {
        "Call_Failure": 0,
        "Complaints": 0,
        "Subscription_Length": 24,
        "Charge_Amount": 50,
        "Seconds_of_use": 300,
        "Frecuency_of_use": 10,
        "Frecuency_of_SMS": 5,
        "Distinct_Called_Numbers": 3,
        "Age_group": 2,
        "Tariff_Plan": 1,
        "Status": 1,
        "Age": 35,
        "Customer_Value": 200.0
    }
    
    sample_churn = {
        "Call_Failure": 2,
        "Complaints": 3,
        "Subscription_Length": 6,
        "Charge_Amount": 20,
        "Seconds_of_use": 100,
        "Frecuency_of_use": 3,
        "Frecuency_of_SMS": 1,
        "Distinct_Called_Numbers": 1,
        "Age_group": 1,
        "Tariff_Plan": 2,
        "Status": 0,
        "Age": 22,
        "Customer_Value": 50.0
    }

    sample_Data_churn = {
        "Call_Failure": 2,
        "Complaints": 0,
        "Subscription_Length": 41,
        "Charge_Amount": 0,
        "Seconds_of_use": 628,
        "Frecuency_of_use": 13,
        "Frecuency_of_SMS": 17,
        "Distinct_Called_Numbers": 5,
        "Age_group": 4,
        "Tariff_Plan": 1,
        "Status": 2,
        "Age": 45,
        "Customer_Value": 58.525
    }

    sample_Data_no_churn = {
        "Call_Failure": 8,
        "Complaints": 0,
        "Subscription_Length": 38,
        "Charge_Amount": 0,
        "Seconds_of_use": 4370,
        "Frecuency_of_use": 71,
        "Frecuency_of_SMS": 5,
        "Distinct_Called_Numbers": 17,
        "Age_group": 3,
        "Tariff_Plan": 1,
        "Status": 1,
        "Age": 30,
        "Customer_Value": 197.64
    }
    
    url = "http://127.0.0.1:8000/predict_log"
    
    response_no_churn = requests.post(url, json=sample_no_churn)
    logger.info(f"Respuesta para cliente sin churn: {response_no_churn.json()}")
    
    response_churn = requests.post(url, json=sample_churn)
    logger.info(f"Respuesta para cliente con churn: {response_churn.json()}")

    response_churn_org = requests.post(url, json=sample_Data_churn)
    logger.info(f"Respuesta orginal cliente con churn: {response_churn_org.json()}")

    response_no_churn_org = requests.post(url, json=sample_Data_no_churn)
    logger.info(f"Respuesta original cliente sin churn: {response_no_churn_org.json()}")

test_api_log()

2025-03-28 16:03:08,309 - INFO - Respuesta para cliente sin churn: {'Predicción de Churn Random Forest': 0, 'Probabilidad de Churn Random Forest': 0.17}
2025-03-28 16:03:08,328 - INFO - Respuesta para cliente con churn: {'Predicción de Churn Random Forest': 1, 'Probabilidad de Churn Random Forest': 0.9}
2025-03-28 16:03:08,353 - INFO - Respuesta orginal cliente con churn: {'Predicción de Churn Random Forest': 1, 'Probabilidad de Churn Random Forest': 0.82}
2025-03-28 16:03:08,374 - INFO - Respuesta original cliente sin churn: {'Predicción de Churn Random Forest': 0, 'Probabilidad de Churn Random Forest': 0.0}
2025-03-28 16:03:08,386 - INFO - Respuesta para cliente sin churn: {'Predicción de Churn Regresión Logistica': 0, 'Probabilidad de Churn Regresión Logistica': 0.0}
2025-03-28 16:03:08,395 - INFO - Respuesta para cliente con churn: {'Predicción de Churn Regresión Logistica': 1, 'Probabilidad de Churn Regresión Logistica': 0.87}
2025-03-28 16:03:08,404 - INFO - Respuesta orginal cli